In [2]:
# load data into dataframe

import numpy as np
import pandas as pd

ui_data = pd.read_json('../data/australian_users_items_fixed.json')
ur_data = pd.read_json('../data/australian_user_reviews_fixed.json')


In [116]:
ui_data.drop_duplicates(subset ="user_id", keep = False, inplace = True)
ur_data.drop_duplicates(subset ="user_id", keep = False, inplace = True)
ui_data.dropna(subset = ["user_id"], inplace=True)
ur_data.dropna(subset = ["user_id"], inplace=True)
print(len(ui_data))
print(len(ur_data))

86953
25176


In [117]:
# creating output format

ratings = {'user_id': [],
           'item_id': [],
           'rating': []
          } 


#rating criteria:

'''
Playtime:
>= 30 +1,
> 300 +1,
> 1000 +1

Recommend:
recommend +1
not recommend -1
'''

'\nPlaytime:\n>= 30 +1,\n> 300 +1,\n> 1000 +1\n\nRecommend:\nrecommend +1\nnot recommend -1\n'

In [118]:
# function for extracting item_id

def extract_item_ids(reviews):
    item_ids = []
    for i in range(len(reviews)):
        item_ids.append(reviews[0][i]['item_id'])
    return item_ids

In [119]:
# compute rating for each user-item pair

for u in range(len(ui_data)):
    
    user = ui_data.iloc[u]
    
    for num in range(user['items_count']):
        
        rating = 1
        ratings['user_id'].append(user['user_id'])
        ratings['item_id'].append(user['items'][num]['item_id'])
        
        
        if 30 <= user['items'][num]['playtime_forever'] < 300:
            rating += 1
        elif 300 <= user['items'][num]['playtime_forever'] < 1000:
            rating += 2
        elif 1000 <= user['items'][num]['playtime_forever']:
            rating += 3
        else:
            rating += 0
        
        if user['user_id'] in ur_data['user_id']:
            
            reviews = ur_data.loc[ur_data['user_id'] == user['user_id']]['reviews']
            
            if user['items'][num]['item_id'] in extract_item_ids(reviews):
            
                review = reviews['item_id' == user['items'][num]['item_id']]
                
                if review['recommend'] == True:
                    rating += 1
                elif review['recommend'] == False:
                    rating -= 1
                else:
                    ratings['rating'].append(rating)
                    continue
            else:
                ratings['rating'].append(rating)
                continue
        else:
            ratings['rating'].append(rating)
            continue

In [120]:
# output the result to a .csv file

output_df = pd.DataFrame(ratings, columns = ['user_id','item_id','rating']) # forming the dataframe

output_df.to_csv(r'/Users/brian80433/Desktop/256Project/data/ratings.csv', index = False, header=True) # output file

